In [1]:
from scipy.spatial import Delaunay, ConvexHull
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from mplsoccer import VerticalPitch
import pandas as pd
import numpy as np
from source_preprocessing import *
from source_clustering import *
from source_plotting import * 
import xml.etree.ElementTree as et
import math
import json
from datetime import datetime
import copy
from sys import intern
import pickle
import networkx as nx
from itertools import groupby
import operator
from sympy import Point, Line, pi

In [2]:
class Match:
    def __init__(self, filePath):
        match = et.parse(filePath).getroot()[0]

        self.matchID      = int(match.attrib['id'])
        self.matchNr      = int(match.attrib['matchNumber'])
        self.date         = match.attrib['dateMatch']
        self.stadiumID    = int(match[1].attrib['id'])
        self.stadiumName  = match[1].attrib['name']
        self.pitchLength  = int(match[1].attrib['pitchLength'])
        self.pitchWidth   = int(match[1].attrib['pitchWidth'])
        self.phases       = [Phase(phase) for phase in match[2]]
        self.frames       = [Frame(frame) for frame in match[3]]

        self.removeExcessFrames()
        
    def removeExcessFrames(self):
        keep = []
        for frame in self.frames:
            for phase in self.phases:
                if frame.time >= phase.start and frame.time <= phase.end:
                    keep.append(frame)
                    break

        self.frames = keep


class Phase:
    def __init__(self, phase):
        self.start       = phase.attrib['start']
        self.end         = phase.attrib['end']
        self.leftTeamID  = int(phase.attrib['leftTeamID'])
        
class Frame:
    def __init__(self, frame):
        self.time            = frame.attrib['utc']
        self.ballInPlay      = frame.attrib['isBallInPlay']
        self.ballPossession  = frame.attrib['ballPossession']
        self.trackingObjs    = [TrackingObj(obj) for obj in frame[0]]
    
class TrackingObj:
    def __init__(self, obj):
        self.type      = obj.attrib['type']
        self.id        = obj.attrib['id']
        self.x         = int(obj.attrib['x'])
        self.y         = int(obj.attrib['y'])
        self.sampling  = obj.attrib['sampling']


In [18]:
path_data = "data/Italy v Spain.xml"
match = Match(path_data)

In [27]:
match.frames[0].trackingObjs[1].id

'250002704'

In [7]:
def df_event_data(path):
    rd = ""
    with open(path, 'r', encoding="UTF-8") as f:
        rd = f.read()
    obj = json.loads(rd)
    df = pd.json_normalize(obj)
    return df


In [5]:
def get_possesion_team(df_event, match, frame):
    if match.frames[frame].ballInPlay == "0":
        return -1
    
    local_time = match.frames[frame].time[-12:-4]
    period_track = 0
    if local_time >=  match.phases[0].start[-12:-4] and local_time <=   match.phases[1].start[-12:-4]:
        begin_time = match.frames[0].time[-12:-4]
        period_track = 1
    elif local_time >=  match.phases[1].start[-12:-4] and local_time <=   match.phases[2].start[-12:-4]:
        begin_time = match.frames[1].time[-12:-4]
        period_track = 2
    elif local_time >=  match.phases[2].start[-12:-4] and local_time <=   match.phases[3].start[-12:-4]:
        begin_time = match.frames[2].time[-12:-4]
        period_track = 3
    elif local_time >=  match.phases[3].start[-12:-4] and local_time <=   match.phases[4].start[-12:-4]:
        begin_time = match.frames[3].time[-12:-4]
        period_track = 4
    elif local_time >=  match.phases[4].start[-12:-4]:
        begin_time = match.frames[4].time[-12:-4]
        period_track = 5
    
    FMT = '%H:%M:%S'
    time_track = (datetime.strptime(local_time, FMT) - datetime.strptime(begin_time, FMT)).seconds
    possession_team = -1
    for e in range(len(df_event)-1):
        period_event = df_event.loc[e]["period"]
        if period_event == period_track:
            time_event_b = (datetime.strptime(df_event.loc[e]["timestamp"][:8], FMT) - datetime.strptime("00:00:00", FMT)).seconds
            time_event_e = (datetime.strptime(df_event.loc[e+1]["timestamp"][:8], FMT) - datetime.strptime("00:00:00", FMT)).seconds
            if time_event_b <= time_track and time_track <= time_event_e:
                possession_team = df_event.loc[e]["possession_team.name"]
                break
    
    return possession_team

In [6]:
def read_player_name_id(path = "data/playerids.csv"):
    df = pd.read_csv(path)
    return df

In [7]:
def get_player_name_id(match, df_player_name_id, keeper_id_home = '', keeper_id_away = ''):

    home_players = {}
    away_players = {}
    for f in match.frames:
        for t in f.trackingObjs:
            if t.type == '0'and t.id != keeper_id_home:
                if t.id not in list(home_players.keys()):
                    home_players[t.id] = df_player_name_id[df_player_name_id["uefa_id"] == int(t.id)]["uefa_name"].item()
            elif t.type == '1' and t.id != keeper_id_away:
                if t.id not in list(away_players.keys()):
                    away_players[t.id] = df_player_name_id[df_player_name_id["uefa_id"] == int(t.id)]["uefa_name"].item()

    return home_players, away_players

In [8]:
def get_coords(match, keeper_id_home = '', keeper_id_away = ''):
    home_coords = []
    away_coords = []
    for f in match.frames:
        home_temp = np.zeros((10,2))
        away_temp = np.zeros((10,2))
        home_idx = 0
        away_idx = 0
        # finland vs russia:  '1907188' - '250052963'
        # belgium vs russia : '250011668' - '250052963'
        # italy vs spain : '250078922' - '250076116'
        # germany vs hungary :   '97923'  - '102420'
        # wales vs denmark: '250101045'- '74954'
        for t in f.trackingObjs:
            if t.type == '0' and t.id != keeper_id_home:
                home_temp[home_idx,0] = t.x
                home_temp[home_idx,1] = t.y
                home_idx += 1
            elif t.type == '1' and t.id != keeper_id_away:
                if away_idx < 10:
                    away_temp[away_idx,0] = t.x
                    away_temp[away_idx,1] = t.y
                    away_idx += 1
        home_coords.append(home_temp)
        away_coords.append(-away_temp)
    return home_coords, away_coords

In [9]:
def get_coords_all(match, keeper_id_home = '', keeper_id_away = ''):
    home_coords = []
    home_coords_orig = []
    away_coords = []
    away_coords_orig = []
    ph = []
    pa = []
    i = 0
    for f in match.frames:
        home_temp = np.zeros((10,2))
        away_temp = np.zeros((10,2))
        home_idx = 0
        away_idx = 0
        # finland vs russia:  '1907188' - '250052963'
        # belgium vs russia : '250011668' - '250052963'
        # italy vs spain : '250078922' - '250076116'
        # germany vs hungary :   '97923'  - '102420'
        # wales vs denmark: '250101045'- '74954'
        home_dir = 0
        for t in f.trackingObjs:
            if t.type == '0' and t.id != keeper_id_home:
                if home_idx < 10:
                    home_temp[home_idx,0] = t.x
                    home_temp[home_idx,1] = t.y
                    home_idx += 1
            elif t.type == '1' and t.id != keeper_id_away:
                if away_idx < 10:
                    away_temp[away_idx,0] = t.x
                    away_temp[away_idx,1] = t.y
                    away_idx += 1
            elif t.id == keeper_id_home:
                home_dir = t.x

        home_coords_orig.append(home_temp)
        away_coords_orig.append(away_temp)

        if home_dir >= 0:
            home_coords.append(home_temp)
            away_coords.append(-away_temp)
            ph.append(i)
        else:
            home_coords.append(-home_temp)
            away_coords.append(away_temp)
            pa.append(i)
        i +=1
    return home_coords, away_coords, home_coords_orig, away_coords_orig

In [10]:
class Vertex:
    def __init__(self, index, loc):
        self.index = index
        self.loc = loc

    def __repr__(self):
        return #f"{self.id}"

class Edge:
    def __init__(self, vertex_from, vertex_to):
        self.vertex_from = vertex_from
        self.vertex_to = vertex_to
        self.triangles = []

    @property
    def on_convex(self):
        """Whether it is interior or convex edge"""
        return 0 if len(self.triangles) == 2 else 1

    def add_triangle(self, triangle):
        self.triangles.append(triangle)

    def __eq__(self, other):
        if self.vertex_from == other.vertex_from and self.vertex_to == other.vertex_to:
            return True
        elif (
            self.vertex_from == other.vertex_to and self.vertex_to == other.vertex_from
        ):
            return True
        else:
            return False

    def __repr__(self):
        return f"({self.vertex_from},{self.vertex_to})"

class DT:
    """
    Delanuary Triangualtion Class to add tolerance and stability attributes to the edges
    """

    roundness = 2

    def __init__(self, player_frames):
        self.vertices = []

        for index, player_frame in enumerate(player_frames):
            vertex = Vertex(index, player_frame)
            self.vertices.append(vertex)
        self.triangles = []
        self.edges = []
        for simplex in Delaunay(self._get_locs()).simplices:
            triangle = [
                self.vertices[simplex[0]],
                self.vertices[simplex[1]],
                self.vertices[simplex[2]],
            ]
            for i in range(3):
                vertex_from = self.vertices[simplex[i]]
                vertex_to = self.vertices[simplex[(i + 1) % 3]]
                edge = Edge(vertex_from, vertex_to)

                for index, pre_edge in enumerate(self.edges):
                    if pre_edge == edge:
                        pre_edge.add_triangle(triangle)
                        self.edges[index] = pre_edge
                        break
                else:
                    edge.add_triangle(triangle)
                    self.edges.append(edge)

            self.triangles.append(triangle)

        for index, edge in enumerate(self.edges):
            edge_with_tolerance = self._calc_edge_tolerance(edge)
            edge_with_tolerance_stability = self._calc_edge_stability(
                edge_with_tolerance
            )
            self.edges[index] = edge_with_tolerance_stability

    def _get_other_vertices(self, edge):
        vertex_i = edge.vertex_from
        vertex_j = edge.vertex_to
        return list(
            {vertex for triangle in edge.triangles for vertex in triangle}
            - {vertex_i, vertex_j}
        )

    def _get_locs(self):
        return np.array([vertex.loc for vertex in self.vertices])

    def _calc_perpendicular(self, vertex_a, vertex_b):
        p_a = Point(vertex_a.loc[0], vertex_a.loc[1])
        p_b = Point(vertex_b.loc[0], vertex_b.loc[1])
        p_mid_ab = (p_a + p_b) / 2
        return Line(p_a, p_b).perpendicular_line(p_mid_ab)

    def _calc_edge_tolerance(self, edge):
        vertex_i = edge.vertex_from
        vertex_j = edge.vertex_to
        other_vertices = self._get_other_vertices(edge)

        if len(edge.triangles) == 2:
            perpendicular_ij = self._calc_perpendicular(vertex_i, vertex_j)

            vertex_k, vertex_l = other_vertices

            perpendicular_kl = self._calc_perpendicular(vertex_k, vertex_l)

            center_point = perpendicular_ij.intersection(perpendicular_kl)[0]

            radiuses = []
            for vertex in [vertex_i, vertex_j, vertex_k, vertex_l]:
                radiuses.append(center_point.distance(vertex.loc))
            radiuses.sort()
            edge.tolerance = float(
                ((radiuses[-1] - radiuses[0]) / 2).round(DT.roundness)
            )

        else:
            other_vertex = other_vertices[0]
            point_i = Point(vertex_i.loc[0], vertex_i.loc[1])
            point_j = Point(vertex_j.loc[0], vertex_j.loc[1])
            point_other = Point(other_vertex.loc[0], other_vertex.loc[1])
            edge.tolerance = round(
                float(point_other.distance((point_i + point_j) / 2) / 2), DT.roundness
            )
        return edge

    def _calc_angle_between_three_points(self, pivot_vertex, vertex_1, vertex_2):
        """
        This function calculates the smallest angle created by <vertex_1, pivot_vertex, vertex_2>
        """
        return Line(pivot_vertex.loc, vertex_1.loc).angle_between(
            Line(pivot_vertex.loc, vertex_2.loc)
        )

    def _calc_edge_stability(self, edge):
        vertex_i = edge.vertex_from
        vertex_j = edge.vertex_to

        angles = 0
        for vertex in self._get_other_vertices(edge):
            angles += self._calc_angle_between_three_points(vertex, vertex_i, vertex_j)

        edge.stability = round(math.degrees(pi - angles), DT.roundness)
        return edge
    
    def plot(self, frame_number=np.nan):
        fig, ax = plt.subplots(figsize=(6.8, 10.5))
        plt.triplot(
            self._get_locs()[:, 1],
            self._get_locs()[:, 0],
            [
                [vertices[0].index, vertices[1].index, vertices[2].index]
                for vertices in self.triangles
            ],
        )
        plt.plot(self._get_locs()[:, 1], self._get_locs()[:, 0], "o", color="blue")
        #for vertex in self.vertices:
        #    plt.annotate(f"{vertex.id}", (vertex.loc[1], vertex.loc[0]))

        for edge in self.edges:
            vertex_i = edge.vertex_from
            vertex_j = edge.vertex_to
            point_i = Point(vertex_i.loc[0], vertex_i.loc[1])
            point_j = Point(vertex_j.loc[0], vertex_j.loc[1])

            mid_point = (point_i + point_j) / 2

            plt.annotate(
                f"({edge.tolerance:.2f}, {edge.stability:.2f}°)",
                (mid_point[1], mid_point[0]),
                horizontalalignment="center",
            )
        ax = plt.gca()
        ax.set_aspect("equal", adjustable="box")
        if not np.isnan(frame_number):
            plt.title(f"Frame {frame_number}")
        #plt.savefig(Path(f"output/{frame_number}.png"), bbox_inches="tight")
        #plt.show()
        #plt.clf()
        plt.close()



In [11]:
def dot(vA, vB):
    return vA[0]*vB[0]+vA[1]*vB[1]

def ang(lineA, lineB):
    # Get nicer vector form
    vA = [(lineA[0][0]-lineA[1][0]), (lineA[0][1]-lineA[1][1])]
    vB = [(lineB[0][0]-lineB[1][0]), (lineB[0][1]-lineB[1][1])]
    # Get dot prod
    dot_prod = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_prod/magA/magB
    # Get angle in radians and then convert to degrees
    res = dot_prod/magB/magA
    if res < -1:
        res = -1
    elif res > 1:
        res = 1
    angle = math.acos(res)
    # Basically doing angle <- angle mod 360
    ang_deg = math.degrees(angle)%360
    
    if ang_deg-180>=0:
        # As in if statement
        return 360 - ang_deg
    else: 
        
        return ang_deg

In [12]:
def get_edge_info(match, coords, frame):
    frame_edge_infos = []
    mus  = coords // 100
    mus = np.asarray([[pair[1], -pair[0]] for pair in mus])
    dt = DT(player_frames = mus)
    dt.plot(frame)
    for edge in dt.edges:
        dic = {}
        dic["edge"] = (list(edge.vertex_to.loc), list(edge.vertex_from.loc))
        dic["tolerance"] = edge.tolerance
        dic["stability"] = edge.stability
        dic["on_convex"] = edge.on_convex
        frame_edge_infos.append(dic)
    return frame_edge_infos, mus

In [12]:
def adjust_chull_nodes_new(chull):
    return list(map(lambda el:list(el), chull))

In [13]:
def extract_edges(edge_info):
    edges = []
    for e in edge_info:
        edges.append(e["edge"])
    return edges

In [14]:
def get_internal_nodes_new(nodes, chull_nodes):
    internal_nodes = []
    new_nodes = []
    for n in nodes:
        new_nodes.append(list(n))
        if list(n) not in chull_nodes:
            internal_nodes.append(list(n))
    return new_nodes, internal_nodes

In [15]:
def get_edge_categories(edges, internal_n, chull_n):

    chull_edges = []
    internal_edges = []
    for e in edges:
        if e[0] in chull_n and e[1] in chull_n:
            continue
        else:
            internal_edges.append(e)

    for c in range(len(chull_n)-1):
        chull_e = [chull_n[c],chull_n[c+1]]
        if  chull_e not in edges:
            edges.append(chull_e)
        chull_edges.append(chull_e)
    
    chull_e = [chull_n[-1], chull_n[0]]
    if  chull_e not in edges:
        edges.append(chull_e)
        
    chull_edges.append(chull_e)
    return chull_edges, internal_edges

In [16]:
def get_other_node(edge, internal_edges, internal_nodes):
    n1 = edge[0]
    n2 = edge[1]
    for n in internal_nodes:
        cnt = 0
        internal_edge_to_remove = []
        for e in internal_edges:
            if n1 in e and n in e:
                cnt += 1
                internal_edge_to_remove.append(e)
            if n2 in e and n in e:
                cnt += 1
                internal_edge_to_remove.append(e)
            if cnt == 2:
                return n, internal_edge_to_remove
    return -1, internal_edge_to_remove

def calc_angle_between_three_points(pivot_vertex, vertex_1, vertex_2):
    return Line(pivot_vertex, vertex_1).angle_between(
        Line(pivot_vertex, vertex_2)
    )
def calc_edge_stability(edge, internal_edges, internal_nodes):
    n1 = edge[0]
    n2 = edge[1]
    n3, internal_edge_to_remove =  get_other_node(edge, internal_edges, internal_nodes)
    if n3 == -1:
        return 180, -1, -1
    angle = calc_angle_between_three_points(n3, n1, n2)
    stability = round(math.degrees(pi - angle), 2)
    return stability, n3, internal_edge_to_remove

def calc_perpendicular(vertex_a, vertex_b):
        p_a = Point(vertex_a.loc[0], vertex_a.loc[1])
        p_b = Point(vertex_b.loc[0], vertex_b.loc[1])
        p_mid_ab = (p_a + p_b) / 2
        return Line(p_a, p_b).perpendicular_line(p_mid_ab)

def calc_edge_tolerance(edge, n3):
        n1 = edge[0]
        n2 = edge[1]
        if n3 == -1:
            return 180

        point_i = Point(n1[0], n1[1])
        point_j = Point(n2[0], n2[1])
        point_other = Point(n3[0], n3[1])
        tolerance = round(
            float(point_other.distance((point_i + point_j) / 2) / 2), DT.roundness
        )
        
        return tolerance

def remove_unstable_chull_edges(chull_edges, internal_edges, internal_nodes, chull_nodes, edges, edge_stability_level = 15, edge_tolerance_level = 0.5):
    if len(internal_nodes) == 0:
        return chull_edges, internal_nodes

    removed_edges = []
    c_chull_edges = []
    for c in chull_edges:
        c_chull_edges.append(c)
    
    for e in c_chull_edges:
        stability, n3, internal_edge_to_remove = calc_edge_stability(e, internal_edges, internal_nodes)
        #tolerance = calc_edge_tolerance(e, n3)
        if stability > edge_stability_level:
            if e not in edges:
                edges.append(e)
        else:
            removed_edges.append(e)
            if e in edges:
                edges.remove(e)
            if e in chull_edges:
                chull_edges.remove(e)
                chull_edges.append([e[0], n3])
                chull_edges.append([n3, e[1]])
            if n3 not in chull_nodes:
                chull_nodes.append(n3)
            internal_edges.remove(internal_edge_to_remove[0])
            internal_edges.remove(internal_edge_to_remove[1])
            internal_nodes.remove(n3)                
    return removed_edges, chull_edges, internal_edges, internal_nodes, chull_nodes, edges

In [17]:
def get_isolate_nodes_bisector(chull_nodes, chull_edges, m_tangent_val = 2.25):
    avg_x = sum(np.array(chull_nodes).T[0]) / len(chull_nodes)
    left_iso_nodes = []
    right_iso_nodes = []
    for n in chull_nodes:
        edge1 = [n]
        edge2 = [n]  
        for e in chull_edges:
            if e[0] == n:
                if len(edge1) == 1:
                    edge1.append(e[1])
                else:
                    edge2.append(e[1])
                    break
            if e[1] == n:
                if len(edge1) == 1:
                    edge1.append(e[0])
                else:
                    edge2.append(e[0])
                    break
        
        vec1x = edge1[1][0] - edge1[0][0]
        vec1y = edge1[1][1] - edge1[0][1]
        vec2x = edge2[1][0] - edge2[0][0]
        vec2y = edge2[1][1] - edge2[0][1]
        len1 =  (vec1x**2 + vec1y**2) ** 0.5 
        len2 =  (vec2x**2 + vec2y**2) ** 0.5 
        bx = vec1x/len1 + vec2x/len2
        by = vec1y/len1 + vec2y/len2
        bisec = [n, [n[0] + bx, n[1] + by]]
        m_tangent = 0
        if (bisec[1][0] - bisec[0][0]) != 0:
            m_bisec = (bisec[1][1] - bisec[0][1]) / (bisec[1][0] - bisec[0][0])
            if m_bisec == 0:
                m_tangent = 9999
            else:
                m_tangent = -1/m_bisec
        if abs(m_tangent) >= m_tangent_val: # 60 degrees
            if n[0] > avg_x:
                right_iso_nodes.append(n)
            else:
                left_iso_nodes.append(n)
                
    return left_iso_nodes, right_iso_nodes, avg_x

In [18]:
def get_lower_upper_nodes(internal_nodes, chull_nodes, left_iso_nodes, right_iso_nodes):
    avg_y = 0
    lower_nodes = []
    upper_nodes = []
    if len(internal_nodes) > 1:
        avg_y = sum(np.array(internal_nodes).T[1]) / len(internal_nodes)
    else:
        avg_y = sum(np.array(chull_nodes).T[1]) / len(chull_nodes)
    
    for c in chull_nodes:
        if c[1] < avg_y:
            if c not in left_iso_nodes and c not in right_iso_nodes:
                lower_nodes.append(c)
        else:
            if c not in left_iso_nodes and c not in right_iso_nodes:
                upper_nodes.append(c)
    return lower_nodes, upper_nodes
        

In [19]:
def get_edge_between_internal_nodes(internal_e, internal_n):
    between_internal_edges = []
    internal_to_boundary = []
    for e in internal_e:
        if e[0] in internal_n and e[1] in internal_n:
            between_internal_edges.append(e)
        else:
            internal_to_boundary.append(e)
    return between_internal_edges, internal_to_boundary

In [20]:
def get_lower_upper_other_edges(chull_edges, upper_nodes, lower_nodes, left_iso_nodes, right_iso_nodes):
    upper_edges = []
    lower_edges = []
    other_edges = []
    for c in chull_edges:
        if c[0] in upper_nodes and c[1] in upper_nodes:
            upper_edges.append(c)
        elif c[0] in lower_nodes and c[1] in lower_nodes:
            lower_edges.append(c)
        else:
            other_edges.append(c)
    return lower_edges, upper_edges, other_edges

In [21]:
def get_formation(lower_nodes, left_iso_nodes, internal_nodes, right_iso_nodes, upper_nodes):
    return (len(lower_nodes), len(left_iso_nodes), len(internal_nodes), len(right_iso_nodes), len(upper_nodes))

In [22]:
def get_player_names_coords(match, orig_coords, coords, team_players, frame):    
    f = match.frames[frame]
    player_names_coords = {}

    for c in range(len(orig_coords)):
        for t in f.trackingObjs:
            if orig_coords[c][0] == t.x and orig_coords[c][1] == t.y:
                xx = int(coords[c][1] / 100)
                yy =  - int(coords[c][0] / 100)
                player_names_coords[team_players[t.id]] = [xx, yy]
                break
    return player_names_coords

In [23]:
def new_adjust_edges(edges):
    e_xs = edges.get_xdata()
    e_ys = edges.get_ydata()
    lines = []
    tmp = []
    for i in range(len(e_xs)):
        if i%3 == 2:
            lines.append(list(tmp))
            tmp = []
        else:
            tmp.append([e_xs[i], e_ys[i]])
    return lines

In [24]:
def get_player_names_coords_roles(player_names_coords, left_iso_n, right_iso_n, internal_n, upper_new, lower_new, between_internal_edges, h = 0.6, ang_h = 75, h4 = 0.25):
    #LEFT ISOLATE
    #1 left iso
    if len(left_iso_n) == 1:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n[0]))]
        player_names_coords[player_n].append("LM")
    #2 left iso
    if len(left_iso_n) == 2:
        if left_iso_n[0][1] > left_iso_n[1][1]:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n[0]))]
            player_names_coords[player_n].append("LF")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n[1]))]
            player_names_coords[player_n].append("LM")
        else:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n[0]))]
            player_names_coords[player_n].append("LM")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n[1]))]
            player_names_coords[player_n].append("LF")
    #>=3 left iso
    left_iso_n_sorted = sorted(left_iso_n, key=lambda x: x[1])
    if len(left_iso_n) > 2:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n_sorted[0]))]
        player_names_coords[player_n].append("LB")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n_sorted[-1]))]
        player_names_coords[player_n].append("LF")
        for i in range(1, len(left_iso_n_sorted)-1):
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(left_iso_n_sorted[i]))]
            player_names_coords[player_n].append("LM")
    #RIGH ISOLATE  
    #1 right iso
    if len(right_iso_n) == 1:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n[0]))]
        player_names_coords[player_n].append("RM")
    #2 right iso
    if len(right_iso_n) == 2:
        if right_iso_n[0][1] > right_iso_n[1][1]:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n[0]))]
            player_names_coords[player_n].append("RF")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n[1]))]
            player_names_coords[player_n].append("RM")
        else:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n[0]))]
            player_names_coords[player_n].append("RM")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n[1]))]
            player_names_coords[player_n].append("RF")
    #>=3 right iso
    right_iso_n_sorted = sorted(right_iso_n, key=lambda x: x[1])
    if len(right_iso_n) > 2:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n_sorted[0]))]
        player_names_coords[player_n].append("RB")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n_sorted[-1]))]
        player_names_coords[player_n].append("RF")
        for i in range(1, len(right_iso_n_sorted)-1):
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(list(right_iso_n_sorted[i]))]
            player_names_coords[player_n].append("RM")
    #UPPER ENVELOPE      
    upper_n_sorted = sorted(upper_new, key=lambda x: x[0])
    for u in range(len(upper_n_sorted)):
        upper_n_sorted[u] = list(upper_n_sorted[u])
    # 1 upper
    if len(upper_n_sorted) == 1:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[0])]
        player_names_coords[player_n].append("CF")
    # 2 upper
    if len(upper_n_sorted) == 2:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[0])]
        player_names_coords[player_n].append("LCF")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[1])]
        player_names_coords[player_n].append("RCF")
    # 3 upper
    if len(upper_n_sorted) == 3:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[0])]
        player_names_coords[player_n].append("LF")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[-1])]
        player_names_coords[player_n].append("RF")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[1])]
        player_names_coords[player_n].append("CF")
    # > 4 upper
    if len(upper_n_sorted) >= 4:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[0])]
        player_names_coords[player_n].append("LF")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[-1])]
        player_names_coords[player_n].append("RF")
        for i in range(1,len(upper_n_sorted)-1):
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(upper_n_sorted[i])]
            if i == 1:
                player_names_coords[player_n].append("LCF")
            elif i == len(upper_n_sorted) - 2:
                player_names_coords[player_n].append("RCF")
            else:
                player_names_coords[player_n].append("CF")

    #LOWER ENVELOPE 
    lower_n_sorted = list(sorted(lower_new, key=lambda x: x[0]))
    for u in range(len(lower_n_sorted)):
        lower_n_sorted[u] = list(lower_n_sorted[u])
    # 1 lower
    if len(lower_n_sorted) == 1:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[0])]
        player_names_coords[player_n].append("CB")
    # 2 lower
    if len(lower_n_sorted) == 2:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[0])]
        player_names_coords[player_n].append("LCB")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[1])]
        player_names_coords[player_n].append("RCB")
    #3 lower
    if len(lower_n_sorted) == 3:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[0])]
        player_names_coords[player_n].append("LCB")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[-1])]
        player_names_coords[player_n].append("RCB")
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[1])]
        player_names_coords[player_n].append("CB")
    #4 lower
    if len(lower_n_sorted) == 4:
        delta_y = lower_n_sorted[1][1] - lower_n_sorted[0][1]
        delta_x = lower_n_sorted[1][0] - lower_n_sorted[0][0]
        slope1 = -1
        if delta_x == 0:
            slope_left = 9999
        else:
            slope_left = delta_y / delta_x
        
        delta_y = lower_n_sorted[-1][1] - lower_n_sorted[2][1]
        delta_x = lower_n_sorted[-1][0] - lower_n_sorted[2][0]
        slope1 = -1
        if delta_x == 0:
            slope_right = 9999
        else:
            slope_right = delta_y / delta_x
        
        if slope_left > h:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[0])]
            player_names_coords[player_n].append("LCB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[1])]
            player_names_coords[player_n].append("LB")
        else:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[0])]
            player_names_coords[player_n].append("LB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[1])]
            player_names_coords[player_n].append("LCB")

        if -slope_right > h:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[-1])]
            player_names_coords[player_n].append("RCB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[2])]
            player_names_coords[player_n].append("RB")
        else:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[-1])]
            player_names_coords[player_n].append("RB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[2])]
            player_names_coords[player_n].append("RCB")

    # > 4 lower
    if len(lower_n_sorted) > 4:
        delta_y = lower_n_sorted[1][1] - lower_n_sorted[0][1]
        delta_x = lower_n_sorted[1][0] - lower_n_sorted[0][0]
        slope1 = -1
        if delta_x == 0:
            slope_left = 9999
        else:
            slope_left = delta_y / delta_x
        
        delta_y = lower_n_sorted[-1][1] - lower_n_sorted[-2][1]
        delta_x = lower_n_sorted[-1][0] - lower_n_sorted[-2][0]
        slope1 = -1
        if delta_x == 0:
            slope_right = 9999
        else:
            slope_right = delta_y / delta_x
        if slope_left > h:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[0])]
            player_names_coords[player_n].append("LCB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[1])]
            player_names_coords[player_n].append("LB")
        else:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[0])]
            player_names_coords[player_n].append("LB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[1])]
            player_names_coords[player_n].append("LCB")

        if -slope_right > h:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[-1])]
            player_names_coords[player_n].append("RCB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[-2])]
            player_names_coords[player_n].append("RB")
        else:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[-1])]
            player_names_coords[player_n].append("RB")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[-2])]
            player_names_coords[player_n].append("RCB")

        for i in range(2,len(lower_n_sorted)-2):
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(lower_n_sorted[i])]
            if i == 2:
                player_names_coords[player_n].append("LCB")
            elif i == len(lower_n_sorted)-3:
                player_names_coords[player_n].append("RCB")
            else:
                player_names_coords[player_n].append("CB")
    
    #INTERNAL 
    internal_3_type = "other"
    internal_n_sorted = []
    if len(internal_n)  != 0:
        internal_n_sorted = list(sorted(internal_n, key=lambda x: x[0]))
        for u in range(len(internal_n_sorted)):
            internal_n_sorted[u] = list(internal_n_sorted[u])
        
        internal_n_sorted2 = list(sorted(internal_n, key=lambda x: x[1]))
        for u in range(len(internal_n_sorted2)):
            internal_n_sorted2[u] = list(internal_n_sorted2[u])
    # 1 internal
    if len(internal_n_sorted) == 1:
        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[0])]
        player_names_coords[player_n].append("CM")
    # 2 internal
    elif len(internal_n_sorted) == 2:
        if len(between_internal_edges) == 1:
            edge1 = between_internal_edges[0]
            n11= edge1[0]
            n12 = edge1[1]
            delta_y = n12[1] - n11[1]
            delta_x = n12[0] - n11[0]
            slope1 = -1
            if delta_x == 0:
                slope1 = 9999
            else:
                slope1 = abs(delta_y / delta_x)
            if slope1 > h:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                player_names_coords[player_n].append("CDM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                player_names_coords[player_n].append("CAM")
            else:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[0])]
                player_names_coords[player_n].append("LCM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[1])]
                player_names_coords[player_n].append("RCM")
        else:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[0])]
                player_names_coords[player_n].append("LCM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[1])]
                player_names_coords[player_n].append("RCM")
    # 3 internal
    elif len(internal_n_sorted) == 3:
        # 2 edges
        if len(between_internal_edges) == 2:
            internal_3_type = "horizontal_chain"
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[0])]
            player_names_coords[player_n].append("LCM")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[1])]
            player_names_coords[player_n].append("CM")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[-1])]
            player_names_coords[player_n].append("RCM")
            
        # 3 edges
        elif len(between_internal_edges) == 3:
            edge1 = between_internal_edges[0]
            n11= edge1[0]
            n12 = edge1[1]
            delta_y = n12[1] - n11[1]
            delta_x = n12[0] - n11[0]
            slope1 = -1
            if delta_x == 0:
                slope1 = 9999
            else:
                slope1 = abs(delta_y / delta_x)

            edge2 = between_internal_edges[1]
            n21 = edge2[0]
            n22 = edge2[1]
            delta_y = n22[1] - n21[1]
            delta_x = n22[0] - n21[0]
            slope2 = -1
            if delta_x == 0:
                slope2 = 9999
            else:
                slope2 = abs(delta_y / delta_x)

            edge3 = between_internal_edges[2]
            n31 = edge3[0]
            n32 = edge3[1]
            delta_y = n32[1] - n31[1]
            delta_x = n32[0] - n31[0]
            slope3 = -1
            if delta_x == 0:
                slope3 = 9999
            else:
                slope3 = abs(delta_y / delta_x)
            if slope1 > slope2 and slope3 > slope2:
                ang13 = ang(edge1, edge3)
                if ang13 > ang_h:
                    if (internal_n_sorted2[0][0] == n21[0] and internal_n_sorted2[0][1] == n21[1]) or (internal_n_sorted2[0][0] == n22[0] and internal_n_sorted2[0][1] == n22[1]):
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                        player_names_coords[player_n].append("CAM")
                        internal_3_type = "upward_triangle_2CMs"
                        if internal_n_sorted2[0][0] < internal_n_sorted2[1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("LCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RCM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("RCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LCM")
                    else:
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                        player_names_coords[player_n].append("CDM")
                        internal_3_type = "downward_triangle_2CMs"
                        if internal_n_sorted2[1][0] < internal_n_sorted2[-1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("RCM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("LCM")
                else:
                    if (internal_n_sorted2[0][0] == n21[0] and internal_n_sorted2[0][1] == n21[1]) or (internal_n_sorted2[0][0] == n22[0] and internal_n_sorted2[0][1] == n22[1]):
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                        player_names_coords[player_n].append("CAM")
                        internal_3_type = "upward_triangle_2DMs"
                        if internal_n_sorted2[0][0] < internal_n_sorted2[1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("LDM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RDM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("RDM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LDM")
                    else:
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                        player_names_coords[player_n].append("CDM")
                        internal_3_type = "downward_triangle_2AMs"
                        if internal_n_sorted2[1][0] < internal_n_sorted2[-1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LAM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("RAM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RAM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("LAM")

            elif slope2 > slope1 and slope3 > slope1:
                ang23 = ang(edge2, edge3)
                if ang23 > ang_h:
                    if (internal_n_sorted2[0][0] == n11[0] and internal_n_sorted2[0][1] == n11[1]) or (internal_n_sorted2[0][0] == n12[0] and internal_n_sorted2[0][1] == n12[1]):
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                        player_names_coords[player_n].append("CAM")
                        internal_3_type = "upward_triangle_2CMs"
                        if internal_n_sorted2[0][0] < internal_n_sorted2[1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("LCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RCM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("RCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LCM")
                    else:
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                        player_names_coords[player_n].append("CDM")
                        internal_3_type = "downward_triangle_2CMs"
                        if internal_n_sorted2[1][0] < internal_n_sorted2[-1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("RCM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("LCM")
                else:
                    if (internal_n_sorted2[0][0] == n11[0] and internal_n_sorted2[0][1] == n11[1]) or (internal_n_sorted2[0][0] == n12[0] and internal_n_sorted2[0][1] == n12[1]):
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                        player_names_coords[player_n].append("CAM")
                        internal_3_type = "upward_triangle_2DMs"
                        if internal_n_sorted2[0][0] < internal_n_sorted2[1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("LDM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RDM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("RDM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LDM")
                    else:
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                        player_names_coords[player_n].append("CDM")
                        internal_3_type = "downward_triangle_2AMs"
                        if internal_n_sorted2[1][0] < internal_n_sorted2[-1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LAM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("RAM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RAM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("LAM")
                        
            elif slope1> slope3 and slope2 > slope3:
                ang12 = ang(edge1,edge2)
                if ang12 > ang_h:
                    if (internal_n_sorted2[0][0] == n31[0] and internal_n_sorted2[0][1] == n31[1]) or (internal_n_sorted2[0][0] == n32[0] and internal_n_sorted2[0][1] == n32[1]):
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                        player_names_coords[player_n].append("CAM")
                        internal_3_type = "upward_triangle_2CMs"
                        if internal_n_sorted2[0][0] < internal_n_sorted2[1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("LCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RCM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("RCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LCM")
                    else:
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                        player_names_coords[player_n].append("CDM")
                        internal_3_type = "downward_triangle_2CMs"
                        if internal_n_sorted2[1][0] < internal_n_sorted2[-1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("RCM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RCM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("LCM")
                else:
                    if (internal_n_sorted2[0][0] == n31[0] and internal_n_sorted2[0][1] == n31[1]) or (internal_n_sorted2[0][0] == n32[0] and internal_n_sorted2[0][1] == n32[1]):
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                        player_names_coords[player_n].append("CAM")
                        internal_3_type = "upward_triangle_2DMs"
                        if internal_n_sorted2[0][0] < internal_n_sorted2[1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("LDM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RDM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                            player_names_coords[player_n].append("RDM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LDM")
                    else:
                        player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                        player_names_coords[player_n].append("CDM")
                        internal_3_type = "downward_triangle_2AMs"
                        if internal_n_sorted2[1][0] < internal_n_sorted2[-1][0]:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("LAM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("RAM")
                        else:
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                            player_names_coords[player_n].append("RAM")
                            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[-1])]
                            player_names_coords[player_n].append("LAM")

    # 4 internal
    elif len(internal_n_sorted) == 4:
        n13= internal_n_sorted2[3]
        n12 = internal_n_sorted2[2]
        delta_y = n13[1] - n12[1]
        delta_x = n13[0] - n12[0]
        slope23 = -1
        if delta_x == 0:
            slope23 = 9999
        else:
            slope23 = abs(delta_y / delta_x)
        
        n11= internal_n_sorted2[1]
        n10 = internal_n_sorted2[0]
        delta_y = n11[1] - n10[1]
        delta_x = n11[0] - n10[0]
        slope10 = -1
        if delta_x == 0:
            slope10 = 9999
        else:
            slope10 = abs(delta_y / delta_x)

        if slope10 < h4 and slope23 < h4:
            if internal_n_sorted2[0][0] > internal_n_sorted2[1][0]:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                player_names_coords[player_n].append("RDM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                player_names_coords[player_n].append("LDM")
            else:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
                player_names_coords[player_n].append("LDM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                player_names_coords[player_n].append("RDM")
                
            if internal_n_sorted2[2][0] > internal_n_sorted2[3][0]:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[2])]
                player_names_coords[player_n].append("RAM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[3])]
                player_names_coords[player_n].append("LAM")
            else:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[2])]
                player_names_coords[player_n].append("LAM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[3])]
                player_names_coords[player_n].append("RAM")
        else:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[0])]
            player_names_coords[player_n].append("CDM")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[3])]
            player_names_coords[player_n].append("CAM")
            if internal_n_sorted2[1][0] < internal_n_sorted2[2][0]:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                player_names_coords[player_n].append("LCM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[2])]
                player_names_coords[player_n].append("RCM")
            else:
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[1])]
                player_names_coords[player_n].append("RCM")
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted2[2])]
                player_names_coords[player_n].append("LCM")
        
    # > 4 internal 
    else:
        if len(internal_n) > 0:
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[0])]
            player_names_coords[player_n].append("LCM")
            player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[-1])]
            player_names_coords[player_n].append("RCM")
            for i in range(1, len(internal_n_sorted)-1):
                player_n = list(player_names_coords.keys())[list(player_names_coords.values()).index(internal_n_sorted[i])]
                player_names_coords[player_n].append("CM")
            
    return player_names_coords, internal_3_type


In [25]:
def new_frame_config(match, team_coords, frame, team_players, orig_team_coords):

   try:
      nodes = team_coords[frame]
      x = [-int(i[0]/100) for i in nodes]
      y = [int(i[1]/100) for i in nodes]
      nodes = np.asarray([[int(pair[1]/100), -int(pair[0]/100)] for pair in nodes])
      fig = plt.figure(figsize=(8, 8))
      ax = plt.gca()
      ax.set_xlim(-40, 40)
      ax.set_ylim(-50, 50)
      edges,_ = ax.triplot(y, x, color = 'b') 
      edges = new_adjust_edges(edges)
      chull = ConvexHull(points=nodes) 
      chull_nodes = list(chull.points[chull.vertices])
      chull_nodes = adjust_chull_nodes_new(chull_nodes)
      nodes, internal_nodes = get_internal_nodes_new(nodes, chull_nodes)
      chull_edges, internal_edges = get_edge_categories(edges, internal_nodes, chull_nodes)
      while True:
            removed_edges2, chull_edges, internal_edges, internal_nodes, chull_nodes, edges = remove_unstable_chull_edges(chull_edges, internal_edges, internal_nodes, chull_nodes, edges)
            if len(removed_edges2) == 0:
                  break
      left_iso_nodes, right_iso_nodes, avg_x = get_isolate_nodes_bisector(chull_nodes, chull_edges)
      lower_nodes, upper_nodes = get_lower_upper_nodes(internal_nodes, chull_nodes, left_iso_nodes, right_iso_nodes)
      between_internal_edges, internal_to_boundary_edges = get_edge_between_internal_nodes(internal_edges, internal_nodes)
      lower_edges, upper_edges, other_edges = get_lower_upper_other_edges(chull_edges, upper_nodes, lower_nodes, left_iso_nodes, right_iso_nodes)
      formation = get_formation(lower_nodes, left_iso_nodes, internal_nodes, right_iso_nodes, upper_nodes)
      player_names_coords = get_player_names_coords(match, orig_team_coords[frame], team_coords[frame], team_players, frame)
      player_names_coords_roles, internal_3_type = get_player_names_coords_roles(player_names_coords, left_iso_nodes, right_iso_nodes, internal_nodes, upper_nodes, lower_nodes, between_internal_edges)
      for i in list(player_names_coords_roles.keys()):
            if len(player_names_coords_roles[i]) == 2:
                  player_names_coords_roles[i].append('ERR')
            elif len(player_names_coords_roles[i]) < 2:
                  player_names_coords_roles[i] = [0,0,'ERR']
   except:
      internal_nodes = []
      chull_nodes = []
      internal_edges = []
      lower_edges = []
      other_edges = []
      upper_edges = []
      between_internal_edges = []
      internal_to_boundary_edges = []
      formation = (10,0,0,0,0)
      team_players_list = list(team_players.values())
      player_names_coords_roles = {}
      internal_3_type = "ERR"
      for i in team_players_list:
            player_names_coords_roles[i] = [0,0,'ERR']

   plt.close()
   return internal_nodes, chull_nodes, internal_edges, lower_edges, other_edges, upper_edges, between_internal_edges, internal_to_boundary_edges , formation, player_names_coords_roles, internal_3_type

In [26]:
def plot_new_frame(internal_n, chull_n, internal_e, lower_e, vertical_e, upper_e, between_internal_edges, internal_to_boundary_edges ,formation, player_names_coords_roles):
    fig, ax = plt.subplots(figsize=(6.8, 10.5))
    ax.set_facecolor('white')
    draw_pitch(ax, orient_vertical=True, pitch_center=(0, 0))
    
    for e in upper_e:
         plt.plot((e[0][0], e[1][0]), (e[0][1], e[1][1]), alpha = 1, linewidth = 1, color = "red")

    for e in lower_e:
         plt.plot((e[0][0], e[1][0]), (e[0][1], e[1][1]), alpha = 1, linewidth = 1, color = "blue")
    
    for e in between_internal_edges:
         plt.plot((e[0][0], e[1][0]), (e[0][1], e[1][1]), alpha = 1, linewidth = 1, color = "black")
    
    for e in internal_to_boundary_edges:
        plt.plot((e[0][0], e[1][0]), (e[0][1], e[1][1]), alpha = 0.7, linewidth = 1, color = "gray")
    
    for e in vertical_e:
        plt.plot((e[0][0], e[1][0]), (e[0][1], e[1][1]), linestyle = '--' ,alpha = 0.7, linewidth = 1, color = "gray")
    
    for n in chull_n:
        for key, value in player_names_coords_roles.items():
            if value[0] == n[0] and value[1] == n[1]:
                ax.text(n[0]-5, n[1]-3, key, fontsize = 8, color = 'black')
                ax.text(n[0]-2, n[1]+2, value[2], fontsize = 8, color = 'black')
                ax.scatter(n[0], n[1], s=100, color = "black", edgecolors = "black")
                
       
    
    for n in internal_n:
        for key, value in player_names_coords_roles.items():
            if value[0] == n[0] and value[1] == n[1]:
                ax.text(n[0]-5, n[1]-3, key, fontsize = 8, color = 'black')
                ax.text(n[0]-2, n[1]+2, value[2], fontsize = 8, color = 'black')
                ax.scatter(n[0], n[1], s=100, color = "black", edgecolors = "black")
    
    ax.text(-34, 55,"Region Counts: {} - ({} - {} - {}) - {}".format(formation[0], formation[1], formation[2], formation[3], formation[4]), fontsize = 15, color = 'red')

    plt.axis('off')
    plt.show()
    plt.close(fig)

In [27]:
def save_half_formations(formations, file_name = "italy_formations.npy"):
    np.save(file_name, formations)
    return

In [28]:
def save_player_roles(player_roles, file_name = "italy_player_roles.npy"):
    np.save(file_name, player_roles)
    return

In [29]:
def new_get_formations_roles(match, orig_coords, team_coords, team_players):
    formations = []
    player_roles = []
    err_frames = []
    for f in range(100000, len(orig_coords), 1):
        _, _, _, _, _, _, _, _ , formation, player_names_coords_roles, _ =  new_frame_config(match, team_coords, f, team_players, orig_coords)
        if f % 3000 == 0:
            print(f)
        if formation[0] == 10:
            err_frames.append(f)
        formations.append(formation)
        player_roles.append(player_names_coords_roles)
    return np.array(formations), np.array(player_roles), err_frames

In [30]:
def wrapper_home(path_data = "data/Italy v Spain.xml", path_playerids = "data/playerids.csv", keeper_id_home = '250078922', keeper_id_away = '250076116'):
    match = Match(path_data)  
    df_player_name_id = read_player_name_id(path_playerids)
    home_players, _ = get_player_name_id(match, df_player_name_id, keeper_id_home = keeper_id_home, keeper_id_away = keeper_id_away)  
    home_coords_all, _, home_coords_orig, _ = get_coords_all(match, keeper_id_home = keeper_id_home, keeper_id_away = keeper_id_away)
    formations, player_roles, err_frames = new_get_formations_roles(match, home_coords_orig, home_coords_all, home_players)
    return formations, player_roles, err_frames

In [31]:
def wrapper_away(path_data = "data/Italy v Spain.xml", path_playerids = "data/playerids.csv", keeper_id_home = '250078922', keeper_id_away = '250076116'):
    match = Match(path_data)  
    df_player_name_id = read_player_name_id(path_playerids)
    _, away_players = get_player_name_id(match, df_player_name_id, keeper_id_home = keeper_id_home, keeper_id_away = keeper_id_away)  
    _, away_coords_all, _, away_coords_orig = get_coords_all(match, keeper_id_home = keeper_id_home, keeper_id_away = keeper_id_away)
    formations, player_roles, err_frames = new_get_formations_roles(match, away_coords_orig,  away_coords_all, away_players)
    return formations, player_roles, err_frames

In [32]:
path_data = "data/Italy v Spain.xml"
path_data2 = "data/Germany v Hungary.xml"
path_playerids = "data/playerids.csv"
df_player_name_id = read_player_name_id(path_playerids)

In [75]:
match = Match(path_data)  
home_players, away_players = get_player_name_id(match, df_player_name_id, keeper_id_home = '250078922', keeper_id_away = '250076116') 
home_coords_all, away_coords_all, home_coords_orig, away_coords_orig = get_coords_all(match, keeper_id_home = '250078922', keeper_id_away = '250076116')

In [ ]:
frame = 5000
internal_nodes, chull_nodes, internal_edges, lower_edges, other_edges, upper_edges, between_internal_edges, internal_to_boundary_edges , formation, player_names_coords_roles, internal_3_type = new_frame_config(match, away_coords_all, frame, away_players, away_coords_orig)
plot_new_frame(internal_nodes, chull_nodes, internal_edges, lower_edges, other_edges, upper_edges, between_internal_edges, internal_to_boundary_edges,formation, player_names_coords_roles)

In [54]:
match2 = Match(path_data2)  
home_players2, away_players2 = get_player_name_id(match2, df_player_name_id, keeper_id_home = '97923', keeper_id_away = '102420') 
home_coords_all2, away_coords_all2, home_coords_orig2, away_coords_orig2= get_coords_all(match2, keeper_id_home = '97923', keeper_id_away = '102420')

In [ ]:
frame = 14200
internal_nodes2, chull_nodes2, internal_edges2, lower_edges2, other_edges2, upper_edges2, between_internal_edges2, internal_to_boundary_edges2 , formation2, player_names_coords_roles2, internal_3_type2 = new_frame_config(match2, home_coords_all2, frame, home_players2, home_coords_orig2)
plot_new_frame(internal_nodes2, chull_nodes2, internal_edges2, lower_edges2, other_edges2, upper_edges2, between_internal_edges2, internal_to_boundary_edges2, formation2, player_names_coords_roles2)

In [33]:
def str_formations(formations):
    formations_str = []
    for f in formations:
        formations_str.append(str(f))
    return formations_str

In [34]:
def extract_frame_formation_transitions(formations_loaded):
    frame_formation_transitions = pd.DataFrame(columns = ['Previous Formation', 'Current Formation', 'Frame Index'])
    for i in range(len(formations_loaded)-1):
        if i % 5000 == 0:
            print(i)
        frame_formation_transitions = frame_formation_transitions.append({'Previous Formation' : formations_loaded[i], 'Current Formation' : formations_loaded[i+1], 
                                                                          'Frame Index' : i+2000}, ignore_index = True)
    return frame_formation_transitions

In [35]:
def extract_ball_posession(match, formations_loaded):
    possess = []
    for frame in range(2000, len(formations_loaded)+ 2000, 1):
        possession = match.frames[frame].ballPossession
        if possession != "Home" and possession != "Away":
            possess.append(-1)
        else:
            possess.append(1)
    return possess

In [36]:
def get_consective(formations_str):
    cnt = 1
    consecutive_cnt = [cnt]
    for frame in range(1, len(formations_str)-1, 1):
        temp_formation = formations_str[frame]
        if formations_str[frame-1] != temp_formation:
            cnt = 1
            consecutive_cnt.append(cnt)
        else:
            cnt = cnt + 1
            consecutive_cnt.append(cnt)

    return consecutive_cnt


In [37]:
def frame_formation_transitions_csv(formations_loaded, team_name = "italy", path_data = "data/Italy v Spain.xml"):
    match = Match(path_data)
    formations_str = str_formations(formations_loaded)
    frame_formation_transitions = extract_frame_formation_transitions(formations_loaded)
    possessions = extract_ball_posession(match,formations_loaded)
    frame_formation_transitions.insert(len(frame_formation_transitions.columns), "ball_possession", possessions)
    consecutive_cnt =get_consective(formations_str)
    frame_formation_transitions.insert(len(frame_formation_transitions.columns), "time_condensed", consecutive_cnt)
    frame_formation_transitions.to_csv("frame_formation_transitions_"+team_name+"_firsthalf.csv", index=False)

In [38]:
def get_column_names(player_names):
    column_names = []
    for p in player_names:
        column_names.append(p + "_x")
        column_names.append(p + "_y")
        column_names.append(p + "_role")
        column_names.append(p + "_h")
        column_names.append(p + "_v")
        column_names.append(p + "_primary_role")
        column_names.append(p + "_other_roles")
    return column_names

In [277]:
def get_players(all_player_roles_loaded):
    players = []
    for ps in all_player_roles_loaded:
        if len(ps.keys()) <= 10:
            for p in ps.keys():
                if p not in players:
                    players.append(p)
    return players

In [237]:
def get_popular_two_levels(role_per_player, role_dict):
    popular_level = {}
    for r,v  in role_per_player.items():
        popular_level[r] = [-3]
        freqs = {-2:0, -1:0, 0:0, 1:0, 2:0}
        err_cnt = 0
        for p in v: 
            if p[2] == "ERR":
                continue
            else:
                if p[2] in role_dict[-2]:
                    freqs[-2] += p[1] - p[0]
                    err_cnt += 1

                if p[2] in role_dict[-1]:
                    freqs[-1] += p[1] - p[0]
                    err_cnt += 1

                if p[2] in role_dict[0]:
                    freqs[0] += p[1] - p[0]
                    err_cnt += 1

                if p[2] in role_dict[1]:
                    freqs[1] += p[1] - p[0]
                    err_cnt += 1

                if p[2] in role_dict[2]:
                    freqs[2] += p[1] - p[0]
                    err_cnt += 1
        
        if err_cnt == 0:
            popular_level[r] = -3
        else:
            max1 = max(freqs.items(), key=operator.itemgetter(1))
            freqs.pop(max1[0])
            max2 =  max(freqs.items(), key=operator.itemgetter(1))
            freqs.pop(max2[0])
            if max2[1] != 0:
                if max1[1] / max2[1] < 1.5:
                    if abs(max1[0] - max2[0]) != 1:
                        popular_level[r] = int((max1[0] + max2[0])/2)
                    else:
                        popular_level[r] = (max1[0] + max2[0])/2
                else:
                    popular_level[r] = max1[0]
            else:
                popular_level[r] = max1[0]

    return popular_level

In [238]:
def get_dominant_role(popular_level_vertical, popular_level_horizontal, role_dict_horizontal, role_dict_vertical):
    dominant_role = {}
    for k in popular_level_vertical.keys():
        if isinstance(popular_level_vertical[k], int) and isinstance(popular_level_horizontal[k], int):
            if popular_level_vertical[k] == -3 and popular_level_horizontal[k] == -3:
                dominant_role[k] = ["ERR"]
            elif popular_level_horizontal[k] == -1 and popular_level_vertical[k] == -2:
                dominant_role[k] = ["LB", "LM"]
            elif popular_level_horizontal[k] == 1 and popular_level_vertical[k] == -2:
                dominant_role[k] = ["LM", "LF"]
            elif popular_level_horizontal[k] == -1 and popular_level_vertical[k] == 2:
                dominant_role[k] = ["RB", "RM"]
            elif popular_level_horizontal[k] == 1 and popular_level_vertical[k] == 2:
                dominant_role[k] = ["RM", "RF"]
            else:
                dominant_role[k] = list(set(role_dict_horizontal[popular_level_horizontal[k]]).intersection(role_dict_vertical[popular_level_vertical[k]]))
        elif isinstance(popular_level_vertical[k], float) and isinstance(popular_level_horizontal[k], int):
            k1 = int(popular_level_vertical[k]-0.5)
            k2 = int(popular_level_vertical[k]+0.5)
            new_vert = role_dict_vertical[k1] + role_dict_vertical[k2]
            dominant_role[k] = list(set(role_dict_horizontal[popular_level_horizontal[k]]).intersection(new_vert))
        elif isinstance(popular_level_horizontal[k], float) and isinstance(popular_level_vertical[k], int):
            k1 = int(popular_level_horizontal[k]-0.5)
            k2 = int(popular_level_horizontal[k]+0.5)
            new_hor = role_dict_horizontal[k1] + role_dict_horizontal[k2]
            dominant_role[k] = list(set(role_dict_vertical[popular_level_vertical[k]]).intersection(new_hor))
        else:
            k1h = int(popular_level_horizontal[k]-0.5)
            k2h= int(popular_level_horizontal[k]+0.5)
            k1v = int(popular_level_vertical[k]-0.5)
            k2v = int(popular_level_vertical[k]+0.5)
            new_vert = role_dict_vertical[k1v] + role_dict_vertical[k2v]
            new_hor = role_dict_horizontal[k1h] + role_dict_horizontal[k2h]
            dominant_role[k] = list(set(new_vert).intersection(new_hor))
    return dominant_role

In [239]:
def get_popular_role(role_per_player, role_dict):
    popular_role = {}
    for r,v in role_per_player.items():
        popular_role[r] = 'INIT'
        all_roles = []
        for p in v:
            role = p[2]
            all_roles.append(role)

        popular_role[r] = max(set(all_roles), key=all_roles.count)
    return popular_role

In [240]:
def extract_player_roles_transitions(player_roles_loaded, role_dict, dominant_role, popular_role, team_name = "italy"):
    index_values = list(range(2000, 2000 + len(player_roles_loaded)))
    player_names = list(player_roles_loaded[0].keys())
    column_names = get_column_names(player_names)
    frame_player_roles_transitions = pd.DataFrame(index = index_values, columns = column_names)
    for i in range(len(player_roles_loaded)):
        if i % 5000 == 0:
            print(i)
        for key, value in player_roles_loaded[i].items():
            if key in player_names:
                frame_player_roles_transitions[key+"_role"][i+2000] =  value[2]
                #spatial coordinate
                if value[2] == "ERR":
                    frame_player_roles_transitions[key+"_x"][i+2000] =  "-"
                    frame_player_roles_transitions[key+"_y"][i+2000] =  "-"
                else:
                    frame_player_roles_transitions[key+"_x"][i+2000] =  value[0]
                    frame_player_roles_transitions[key+"_y"][i+2000] =  value[1]
                #role enumaration
                hv = role_dict[value[2]]
                frame_player_roles_transitions[key+"_h"][i+2000] =  hv[0]
                frame_player_roles_transitions[key+"_v"][i+2000] =  hv[1]
                
                #primary and secondary roles
                if len(dominant_role[key]) == 2:
                    if popular_role[key] in dominant_role[key]:
                        frame_player_roles_transitions[key+"_primary_role"][i+2000] =  popular_role[key]
                        rest = ""
                        for r in dominant_role[key]:
                            if r != popular_role[key]:
                                rest += r + ", "
                        frame_player_roles_transitions[key+"_other_roles"][i+2000] =  rest[:-2]
                    else:
                        frame_player_roles_transitions[key+"_primary_role"][i+2000] =  dominant_role[key][0]
                        rest = ""
                        for r in dominant_role[key][1:]:
                            rest += r + ", "
                        frame_player_roles_transitions[key+"_other_roles"][i+2000] =  rest[:-2]
                else:
                   frame_player_roles_transitions[key+"_primary_role"][i+2000] =  dominant_role[key][0]
                   frame_player_roles_transitions[key+"_other_roles"][i+2000] =  "-"
    return frame_player_roles_transitions

In [241]:
def frame_player_roles_transitions_csv(player_roles_loaded, role_dict, role_dict_horizontal, role_dict_vertical, team_name = "italy"):
    role_per_player = get_role_per_player(player_roles_loaded)
    popular_level_horizontal = get_popular_two_levels(role_per_player, role_dict_horizontal)
    popular_level_vertical = get_popular_two_levels(role_per_player, role_dict_vertical)
    dominant_role = get_dominant_role(popular_level_vertical, popular_level_horizontal, role_dict_horizontal, role_dict_vertical)
    popular_role = get_popular_role(role_per_player, role_dict)
    frame_player_roles_transitions = extract_player_roles_transitions(player_roles_loaded, role_dict, dominant_role, popular_role, team_name)
    frame_player_roles_transitions.to_csv("frame_player_roles_transitions_"+team_name+"_firsthalf.csv")

In [47]:
roles_vertical = ["LF", "LM", "LB", 
                  "LCF", "LAM", "LCM", "LDM", "LCB", 
                  "CF", "CAM", "CM", "CDM", "CB", 
                  "RCF", "RAM", "RCM", "RDM", "RCB", 
                  "RF", "RM", "RB", 
                  "ERR"]
colors_vertical = ["#8c510a", "#8c510a", "#8c510a", 
                   "#dfc27d", "#dfc27d", "#dfc27d", "#dfc27d", "#dfc27d",
                   "lightgray", "lightgray", "lightgray", "lightgray", "lightgray", #f5f5f5
                   "#80cdc1", "#80cdc1", "#80cdc1", "#80cdc1", "#80cdc1", 
                   "#01665e", "#01665e", "#01665e", 
                   "white"]

roles_horizontal = ["LF", "LCF", "CF", "RCF", "RF",
                    "LAM", "CAM", "RAM", 
                    "LM", "LCM", "CM", "RCM", "RM",
                    "LDM", "CDM", "RDM", 
                    "LB", "LCB", "CB", "RCB", "RB", 
                    "ERR"]

colors_horizontal = ["#b2182b", "#b2182b", "#b2182b", "#b2182b", "#b2182b",
                     "#f4a582", "#f4a582", "#f4a582",
                     "lightgray", "lightgray", "lightgray", "lightgray", "lightgray", #f7f7f7
                     "#92c5de", "#92c5de", "#92c5de", 
                     "#2166ac", "#2166ac", "#2166ac", "#2166ac", "#2166ac",
                     "white"]
                     
role_dict = {"LF": [-2,2], "LCF": [-1,2], "CF": [0,2], "RCF": [1,2], "RF": [2,2],
             "LAM": [-1,1], "CAM": [0,1], "RAM": [1,1],
             "LM": [-2,0], "LCM": [-1,0], "CM": [0,0,], "RCM": [1,0], "RM": [2,0],
             "LDM": [-1,-1], "CDM": [0,-1], "RDM": [1,-1],
             "LB": [-2,-2], "LCB": [-1,-2], "CB": [0,-2], "RCB": [1,-2], "RB": [2,-2], "ERR" : ["-","-"]}

role_dict_horizontal = {-2 : ["LB", "LCB", "CB", "RCB", "RB"], -1: ["LDM", "CDM", "RDM"], 
                      0: ["LM", "LCM", "CM", "RCM", "RM"], 1: ["LAM", "CAM", "RAM"], 2: ["LF", "LCF", "CF", "RCF", "RF"]}

role_dict_vertical = {-2: ["LF", "LM", "LB"], -1: ["LCF", "LAM", "LCM", "LDM", "LCB"], 
                      0: ["CF", "CAM", "CM", "CDM", "CB"], 1: ["RCB", "RDM", "RCM", "RAM", "RCF"], 2: ["RF", "RM", "RB"]}

In [242]:
def get_color_dict_manual(colors, roles):
    color_dict = {}
    for r in range(len(roles)):
         color_dict[roles[r]] = colors[r]
    return color_dict

In [243]:
def get_max_change(role_per_player):
    lens = []
    for i in role_per_player.keys():
        lens.append(len(role_per_player[i]))
    
    return np.max(np.array(lens))

In [244]:
def get_sub_idx(all_player_roles_loaded):
    sub_idx = []
    for i in range(1,len(all_player_roles_loaded)):
        prev_f = set(list(all_player_roles_loaded[i-1].keys()))
        curr_f = set(list(all_player_roles_loaded[i].keys()))
        if prev_f != curr_f and len(prev_f) == 10 and len(curr_f) == 10:
            sub_idx.append(i)
    return sub_idx

In [246]:
def get_all_role_per_players(all_player_roles_loaded, threshold = 1500):
    all_role_per_players = []
    sub_idx = get_sub_idx(all_player_roles_loaded)
    sub_idx.insert(0,0)
    for s in range(len(sub_idx)-1):
        if sub_idx[s+1] - sub_idx[s] > threshold:
            print(sub_idx[s], sub_idx[s+1])
            all_role_per_players.append(get_role_per_player_new(all_player_roles_loaded[sub_idx[s]:sub_idx[s+1]]))
    
    return all_role_per_players

In [247]:
def extract_ball_posession_per_team(match, formations_loaded, team, start_idx = 0):
    possess = []
    in_pos = ""
    out_pos = ""
    if team == "Home":
        in_pos = "Home"
        out_pos = "Away"
    else:
        in_pos = "Away"
        out_pos = "Home"

    for frame in range(start_idx, len(formations_loaded)+ start_idx, 1):
        possession = match.frames[frame].ballPossession
        if possession == in_pos:
            possess.append(1)
        elif possession == out_pos:
            possess.append(-1)
        else:
            possess.append(0)
    return possess

In [248]:
def get_role_per_player_new(all_player_roles_loaded):
    sf = 2000
    role_per_player = {}
    players = get_players(all_player_roles_loaded)
    px = 0
    for p in players:
        started = False
        last_seen = 0
        for i in range(1, len(all_player_roles_loaded), 1):
            if p in all_player_roles_loaded[i].keys() and len(all_player_roles_loaded[i].keys()) <= 10:
                last_seen = i
                if started == False:
                    role_per_player[p] = [[sf + i - 1]]
                    started = True
                temp_role = all_player_roles_loaded[i][p][2]
                try:
                    if all_player_roles_loaded[i-1][p][2] != temp_role:
                        role_per_player[p][-1].append(sf + i) # end time
                        role_per_player[p][-1].append(all_player_roles_loaded[i-1][p][2]) # role in the interval
                        role_per_player[p].append([]) 
                        role_per_player[p][-1].append(sf + i + 1) # start time
                except:
                    px = px + 1
                    #print(px)
                    continue

        role_per_player[p][-1].append(sf + last_seen) # end time
        role_per_player[p][-1].append(all_player_roles_loaded[last_seen][p][2]) # role in the interval
    return role_per_player

In [249]:
def get_role_per_player_new_agg(match, all_player_roles_loaded, team, window_size = 25, poss = "in"):
    possess = extract_ball_posession_per_team(match, all_player_roles_loaded, team)
    if poss == "in":
        poss_interval = 1
    else:
        poss_interval = -1
    players = get_players(all_player_roles_loaded)
    role_per_player = {}
    sub_idx = get_sub_idx(all_player_roles_loaded)
    for p in players:
        roles_in_window = []
        idx = 0
        role_per_player[p] = []
        for i in range(len(all_player_roles_loaded)):
            if  i in sub_idx:
                for c in range(25):
                    role_per_player[p].append([idx, idx + 1, "ERR"])
                    idx += 1
            else:
                if possess[i] != 0: # if ball in play
                    if p in all_player_roles_loaded[i].keys() and len(all_player_roles_loaded[i].keys()) <= 10: #not an errorneous frame and player is in play
                        if i % window_size != 0 or i == 0:
                            if possess[i] != poss_interval: #not in requested Possession
                                roles_in_window.append("ERR")
                            else:
                                roles_in_window.append(all_player_roles_loaded[i][p][2]) #requested possession
                    else:
                        if i % window_size != 0 or i == 0:
                            roles_in_window.append("ERR")

                    if i % window_size == 0 and i != 0: # window is full
                        try:
                            role_for_window = max(set(roles_in_window), key = roles_in_window.count) #get majority role
                            role_per_player[p].append([idx, idx + 1, role_for_window])
                            roles_in_window = []
                        except:
                            role_per_player[p].append([idx, idx + 1, "ERR"])
                            roles_in_window = []
                        idx += 1

    return role_per_player

In [250]:
def get_role_per_player_new_agg_all(match, all_player_roles_loaded, team, window_size = 25):
    possess = extract_ball_posession_per_team(match, all_player_roles_loaded, team)
    players = get_players(all_player_roles_loaded)
    role_per_player = {}
    sub_idx = get_sub_idx(all_player_roles_loaded)
    for p in players:
        roles_in_window = []
        idx = 0
        role_per_player[p] = []
        for i in range(len(all_player_roles_loaded)):
            if  i in sub_idx:
                for c in range(25):
                    role_per_player[p].append([idx, idx + 1, "ERR"])
                    idx += 1
            else:
                if possess[i] != 0: # if ball in play
                    if p in all_player_roles_loaded[i].keys() and len(all_player_roles_loaded[i].keys()) <= 10: #not an errorneous frame and player is in play
                        if i % window_size != 0 or i == 0:
                            roles_in_window.append(all_player_roles_loaded[i][p][2])
                    else:
                        if i % window_size != 0 or i == 0:
                            roles_in_window.append("ERR")

                    if i % window_size == 0 and i != 0: # window is full
                        try:
                            role_for_window = max(set(roles_in_window), key = roles_in_window.count) #get majority role
                            role_per_player[p].append([idx, idx + 1, role_for_window])
                            roles_in_window = []
                        except:
                            role_per_player[p].append([idx, idx + 1, "ERR"])
                            roles_in_window = []
                        idx += 1

    return role_per_player

In [297]:
def get_role_per_player_new_all(match, all_player_roles_loaded, team):
    players = get_players(all_player_roles_loaded)
    role_per_player = {}
    for p in players:
        idx = 0
        role_per_player[p] = []
        cnt = 0
        for i in range(len(all_player_roles_loaded)):
            if p in all_player_roles_loaded[i].keys() and len(all_player_roles_loaded[i].keys()) <= 10: #not an errorneous frame and player is in play
                role_per_player[p].append([idx, idx + 1, all_player_roles_loaded[i][p][2]])
                cnt += 1
            else:
                role_per_player[p].append([idx, idx + 1,"ERR"])
            idx += 1
    return role_per_player

In [298]:
def get_dominant_role_per_interval(all_player_roles, match, role_dict_horizontal, role_dict_vertical, team = "Home"):
    sub_idx = get_sub_idx(all_player_roles)
    sub_idx.insert(0,0)
    sub_idx.append(len(all_player_roles))
    dominant_roles = []
    for i in range(len(sub_idx) - 1):
        role_per_player = get_role_per_player_new_all(match, all_player_roles[sub_idx[i]:sub_idx[i+1]], team) # role per player in given interval
        popular_level_horizontal = get_popular_two_levels(role_per_player, role_dict_horizontal)
        popular_level_vertical = get_popular_two_levels(role_per_player, role_dict_vertical)
        dominant_role = get_dominant_role(popular_level_vertical, popular_level_horizontal, role_dict_horizontal, role_dict_vertical)
        dominant_roles.append(dominant_role)
    return dominant_roles, sub_idx
        

In [77]:
player_roles_home_loaded = np.load("italy_player_roles.npy", allow_pickle=  True)
player_roles_home_loaded_p2 = np.load("italy_player_roles_p2.npy", allow_pickle=  True)
player_roles_home_loaded_p3 = np.load("italy_player_roles_p3.npy", allow_pickle=  True)
player_roles_away_loaded = np.load("spain_player_roles.npy", allow_pickle=  True)
player_roles_away_loaded_p2 = np.load("spain_player_roles_p2.npy", allow_pickle=  True)
player_roles_away_loaded_p3 = np.load("spain_player_roles_p3.npy", allow_pickle=  True)

In [37]:
player_roles2_home_loaded = np.load("germany_player_roles_1.npy", allow_pickle=  True)
player_roles2_home_loaded_p2 = np.load("germany_player_roles_2.npy", allow_pickle=  True)
player_roles2_home_loaded_p3 = np.load("germany_player_roles_3.npy", allow_pickle=  True)
player_roles2_away_loaded = np.load("hungary_player_roles_1.npy", allow_pickle=  True)
player_roles2_away_loaded_p2 = np.load("hungary_player_roles_2.npy", allow_pickle=  True)
player_roles2_away_loaded_p3 = np.load("hungary_player_roles_3.npy", allow_pickle=  True)

In [78]:
all_player_roles_home_loaded = list(player_roles_home_loaded) + list(player_roles_home_loaded_p2)[2000:] + list(player_roles_home_loaded_p3)
all_player_roles_away_loaded = list(player_roles_away_loaded) + list(player_roles_away_loaded_p2) + list(player_roles_away_loaded_p3)

In [38]:
all_player_roles2_home_loaded = list(player_roles2_home_loaded) + list(player_roles2_home_loaded_p2) + list(player_roles2_home_loaded_p3)
all_player_roles2_away_loaded = list(player_roles2_away_loaded) + list(player_roles2_away_loaded_p2) + list(player_roles2_away_loaded_p3)

In [79]:
home_roles_in_poss = get_role_per_player_new_agg(match, all_player_roles_home_loaded, "Home", window_size = 50, poss = "in")
home_roles_out_poss = get_role_per_player_new_agg(match, all_player_roles_home_loaded, "Home", window_size = 50, poss = "out")
away_roles_in_poss = get_role_per_player_new_agg(match, all_player_roles_away_loaded, "Away", window_size = 50, poss = "in")
away_roles_out_poss = get_role_per_player_new_agg(match, all_player_roles_away_loaded, "Away", window_size = 50, poss = "out")
home_roles_in_out_poss = get_role_per_player_new_agg_all(match, all_player_roles_home_loaded, "Home", window_size = 50)
away_roles_in_out_poss = get_role_per_player_new_agg_all(match, all_player_roles_away_loaded, "Away", window_size = 50)


In [74]:
home_roles2_in_poss = get_role_per_player_new_agg(match2, all_player_roles2_home_loaded, "Home", window_size = 50, poss = "in")
home_roles2_out_poss = get_role_per_player_new_agg(match2, all_player_roles2_home_loaded, "Home", window_size = 50, poss = "out")
away_roles2_in_poss = get_role_per_player_new_agg(match2, all_player_roles2_away_loaded, "Away", window_size = 50, poss = "in")
away_roles2_out_poss = get_role_per_player_new_agg(match2, all_player_roles2_away_loaded, "Away", window_size = 50, poss = "out")
home_roles2_in_out_poss = get_role_per_player_new_agg_all(match2, all_player_roles2_home_loaded, "Home", window_size = 50)
away_roles2_in_out_poss = get_role_per_player_new_agg_all(match2, all_player_roles2_away_loaded, "Away", window_size = 50)

In [299]:
dominant_roles_home,sub_idx_home = get_dominant_role_per_interval(all_player_roles_home_loaded, match, role_dict_horizontal, role_dict_vertical, team = "Home")
dominant_roles_away,sub_idx_away = get_dominant_role_per_interval(all_player_roles_away_loaded, match, role_dict_horizontal, role_dict_vertical, team = "Away")

In [300]:
dominant_roles2_home,sub_idx2_home = get_dominant_role_per_interval(all_player_roles2_home_loaded, match2, role_dict_horizontal, role_dict_vertical, team = "Home")
dominant_roles2_away,sub_idx2_away = get_dominant_role_per_interval(all_player_roles2_away_loaded, match2, role_dict_horizontal, role_dict_vertical, team = "Away")

In [301]:
def get_csv_per_interval(dominant_roles, sub_idx,  team_name= "Italy", match_name =  "Ita v Esp"):
    index_pairs = []
    for s in range(len(sub_idx) - 1):
        index_pairs.append("Interval_" + str(s) + "-"+ "Interval_" + str(s+1))
    df = pd.DataFrame(dominant_roles)
    df.index = index_pairs
    df.to_csv(team_name+"_"+match_name+".csv")
    return df

In [304]:
dominant_roles_home_df = get_csv_per_interval(dominant_roles_home, sub_idx_home, team_name= "Italy", match_name =  "Ita v Esp")
dominant_roles_away_df = get_csv_per_interval(dominant_roles_away, sub_idx_away, team_name= "Spain", match_name =  "Ita v Esp")

In [305]:
dominant_roles2_home_df = get_csv_per_interval(dominant_roles2_home, sub_idx2_home, team_name= "Germany", match_name =  "Ger v Hun")
dominant_roles2_away_df = get_csv_per_interval(dominant_roles2_away, sub_idx2_away, team_name= "Hungary", match_name =  "Ger v Hun")

In [63]:
def get_ordered_player_names(popular_level_vertical, popular_level_horizontal):
    possible_vals_ver = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]
    possible_vals_hor = [2, 1.5, 1, 0.5, 0, -0.5, -1, -1.5, -2]
    player_names = []
    for ph in possible_vals_hor:
        for pv in possible_vals_ver:
            for k, vv in popular_level_vertical.items():
                if vv == pv and popular_level_horizontal[k] == ph:
                    player_names.append(k)
    return player_names

In [65]:
def plot_player_roles_vertical_horizontal(role_per_player, roles_vertical, colors_vertical, roles_horizontal, colors_horizontal, pos = "in", team_name = "Italy", match_name = "Ita v Esp"):

    popular_level_horizontal = get_popular_two_levels(role_per_player, role_dict_horizontal)
    popular_level_vertical = get_popular_two_levels(role_per_player, role_dict_vertical)
    dominant_role = get_dominant_role(popular_level_vertical, popular_level_horizontal, role_dict_horizontal, role_dict_vertical)
    player_names = get_ordered_player_names(popular_level_vertical, popular_level_horizontal)
    player_names.reverse()
    color_dict_vertical = get_color_dict_manual(colors_vertical, roles_vertical)
    color_dict_horizontal = get_color_dict_manual(colors_horizontal, roles_horizontal)
    width = 0.4
    fig, ax = plt.subplots(figsize=(40,20))
    ax.tick_params(axis="both", labelsize=15)
    ax.set_title("Player Roles " + team_name + " - " + match_name + " - " +pos + " possession", fontsize=30)
    ytick = []
    for i in player_names:
        role_str = ""
        for r in dominant_role[i]:
            role_str += r + "_"
        role_str = role_str[:-1]
        ytick.append(i + "_" + role_str)
    ind = np.arange(len(player_names))
    len_max = get_max_change(role_per_player)
    print(len_max)
    for r in range(len_max):
        lefts = []
        rights = []
        colors_ver = []
        colors_hor = []
        roles = []
        for p in player_names:
            if len(role_per_player[p]) - 1 >= r:
                lefts.append(role_per_player[p][r][0])
                rights.append(role_per_player[p][r][1])
                role = role_per_player[p][r][2]
                roles.append(role)
                colors_ver.append(color_dict_vertical[role])
                colors_hor.append(color_dict_horizontal[role])
            else:
                lefts.append(role_per_player[p][-1][0])
                rights.append(role_per_player[p][-1][0])
                roles.append(role_per_player[p][-1][2])
                colors_ver.append("white")
                colors_hor.append("white")
        
        lefts = np.array(lefts)
        rights = np.array(rights)
        if len(lefts) > 0:
            ind = np.array(ind)
            ax.barh(ind, left = lefts, height = width, width = rights - lefts, color= colors_ver, label = roles)
            ax.barh(ind + width, left = lefts, height = width, width = rights - lefts, color = colors_hor, label = roles)
            ax.set(yticks = ind + width, yticklabels= ytick)
        
        if r % 100 == 0:
            print(r)
    
    plt.xlabel("Frames in Corresponding Interval", fontsize = 15)
    plt.ylabel("Players and Roles", fontsize = 15)
    plt.savefig(team_name + "_role_change_timeline_" + pos + "_poss"+  ".png")
    plt.show()
    return

In [66]:
def plot_player_roles_vertical_horizontal_new(role_per_player, roles_vertical, colors_vertical, roles_horizontal, colors_horizontal, pos = "in", team_name = "Italy", match_name = "Ita v Esp"):

    popular_level_horizontal = get_popular_two_levels(role_per_player, role_dict_horizontal)
    popular_level_vertical = get_popular_two_levels(role_per_player, role_dict_vertical)
    player_names = get_ordered_player_names(popular_level_vertical, popular_level_horizontal)
    player_names.reverse()
    color_dict_vertical = get_color_dict_manual(colors_vertical, roles_vertical)
    color_dict_horizontal = get_color_dict_manual(colors_horizontal, roles_horizontal)
    width = 0.4
    fig, ax = plt.subplots(figsize=(40,20))
    ax.tick_params(axis="both", labelsize=15)
    ax.set_title("Player Roles " + team_name + " - " + match_name + " - " +pos + " possession", fontsize=30)
    ytick = []
    for i in player_names:
         ytick.append(i)
    ind = np.arange(len(player_names))
    len_max = get_max_change(role_per_player)
    print(len_max)
    for r in range(len_max):
        lefts = []
        rights = []
        colors_ver = []
        colors_hor = []
        roles = []
        for p in player_names:
            lefts.append(role_per_player[p][r][0])
            rights.append(role_per_player[p][r][1])
            role = role_per_player[p][r][2]
            roles.append(role)
            colors_ver.append(color_dict_vertical[role])
            colors_hor.append(color_dict_horizontal[role])
        
        lefts = np.array(lefts)
        rights = np.array(rights)
        if len(lefts) > 0:
            ind = np.array(ind)
            ax.barh(ind, left = lefts, height = width, width = rights - lefts, color= colors_ver, label = roles)
            ax.barh(ind + width, left = lefts, height = width, width = rights - lefts, color = colors_hor, label = roles)
            ax.set(yticks = ind + width, yticklabels= ytick)
        
        if r % 500 == 0:
            print(r)
    
    plt.xlabel("Frames in Corresponding Interval", fontsize = 15)
    plt.ylabel("Players and Roles", fontsize = 15)
    plt.savefig(team_name + "_role_change_timeline_" + pos + "_poss"+  ".png")
    plt.show()
    return

In [ ]:
plot_player_roles_vertical_horizontal_new(away_roles_in_out_poss, roles_vertical, colors_vertical, roles_horizontal, colors_horizontal, pos = "in_out", team_name= "Spain", match_name =  "Ita v Esp")